SQL ЗАПРОСЫ

**Цель исследования:** анализировать информацию из БД о книгах, издательствах, авторах, а также пользовательских обзорах книг, сформулировать ценностное предложение для нового продукта.

## **Создаем коннект к БД.**

In [ ]:
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## **Выводим первые 5 строк для таблиц books, authors, publishers, ratings, reviews.**

In [ ]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [ ]:
query = '''SELECT COUNT(DISTINCT review_id) FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


## **Посчитайте, сколько книг вышло после 1 января 2000 года**

In [ ]:
query_1 = '''SELECT COUNT(DISTINCT book_id) FROM books WHERE EXTRACT(YEAR FROM publication_date) >= 2000;'''
pd.io.sql.read_sql(sql=text(query_1), con = con)

,count
0,821


Количество книг для приложения с 2000 года не выглядит таким уж и большим.

## **Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [ ]:
query_2 = '''SELECT DISTINCT books.book_id,
                  books.title,
                  COUNT(DISTINCT review_id) AS review_count,
                  AVG(rating) AS avg_rating
FROM books LEFT JOIN reviews ON books.book_id=reviews.book_id
LEFT JOIN ratings ON reviews.book_id=ratings.book_id
GROUP BY books.book_id
ORDER BY review_count DESC
;'''

pd.io.sql.read_sql(sql=text(query_2), con = con)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,207,Eat Pray Love,6,3.395833
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,497,Outlander (Outlander #1),6,4.125000
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,NaN
996,221,Essential Tales and Poems,0,NaN
997,387,Leonardo's Notebooks,0,NaN
998,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,NaN


In [ ]:
query = '''SELECT COUNT(DISTINCT review_id) FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Видно, что самыми популярными книгами чаще являются книги из жанра фантастики, которые были экранизированы (Гарри Поттер, Сумерки). Но, в целом, самое большое количество отзывов - 6-7, что не очень много.

## **Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;**

In [ ]:
query =  '''WITH new_books AS
          (SELECT publisher,
          COUNT(DISTINCT books.book_id) AS book_count
          FROM publishers JOIN books ON publishers.publisher_id=books.publisher_id
          WHERE books.num_pages > 50
          GROUP BY publishers.publisher
          ORDER BY book_count DESC
          LIMIT 1)

      SELECT publisher,
             book_count
        FROM new_books;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,book_count
0,Penguin Books,42


In [ ]:
query =  '''SELECT title
          FROM books JOIN publishers ON books.publisher_id=publishers.publisher_id
          WHERE publisher LIKE '%Penguin Books%'
          '''

pd.io.sql.read_sql(sql=text(query), con = con)

,title
0,A Little Princess
1,A Passage to India
2,A Princess of Mars (Barsoom #1)
3,A Room of One's Own
4,Alexander Hamilton
5,Bleak House
6,Cannery Row
7,Collapse: How Societies Choose to Fail or Succeed
8,Disgrace
9,Don Quixote


Британское издательство, которое выпустило 46 книг, что не является большим количеством, плюс, только одна их этих книг находится в топ-15 книг, которые имеют больше всего оценок (на 15 месте).

## **Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**

In [ ]:
query =  '''WITH good_books AS

(SELECT DISTINCT books.book_id,
        COUNT(ratings.rating) AS rating_count,
        AVG(ratings.rating) AS avg_rating
        FROM books JOIN ratings ON books.book_id=ratings.book_id
        GROUP BY books.book_id
        HAVING COUNT(rating) >= 50)


SELECT authors.author,
       AVG(good_books.avg_rating) AS avg_rating
FROM good_books LEFT JOIN books ON good_books.book_id=books.book_id JOIN authors ON books.author_id=authors.author_id
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1
;'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


In [ ]:
query =  '''SELECT title
          FROM books JOIN authors ON books.author_id=authors.author_id
          WHERE author LIKE '%J.K. Rowling/Mary GrandPré%'
          '''

pd.io.sql.read_sql(sql=text(query), con = con)

,title
0,Harry Potter Boxed Set Books 1-5 (Harry Potte...
1,Harry Potter and the Chamber of Secrets (Harry...
2,Harry Potter and the Half-Blood Prince (Harry ...
3,Harry Potter and the Order of the Phoenix (Har...
4,Harry Potter and the Prisoner of Azkaban (Harr...


Самым популярным автором с самой высокой оценкой книг стала J.K. Rowling, написавшая знаменитого Гарри Поттера. Её книги также имеют одну из самых высоких оценок.

## **Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [ ]:
query =  '''WITH active_users AS
       (SELECT username,
       COUNT(DISTINCT rating_id) AS rating_count
       FROM ratings
       GROUP BY username
       HAVING COUNT(rating_id) > 48),

       users_reviews AS (SELECT username,
       COUNT(DISTINCT review_id) AS review_count
       FROM reviews
       WHERE username IN (SELECT username
                          FROM active_users)
       GROUP BY username)

       SELECT AVG(review_count)
              FROM users_reviews'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


In [ ]:
query =  '''SELECT COUNT(DISTINCT username)
            FROM ratings'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,160


Активные пользователи, которые оставляют более 48 оценок, в среднем пишут 24 отзыва.

## **Вывод**

Пользователи будут активнее, если сервис будет загружать больше книг, похожих на популярные (экранизированные книги из жанра фантастики) из более релевантных изданий.

Об этой проблеме говорят результаты анализа количества загружаемых книг по изданиям, они не совсем по вкусу пользователям. Что касается авторов, то здесь очевидно популярная J.K. Rowling, написавшая Гарри Поттера. Её книги также имеют одну из самых высоких оценок.